# Stock Price Prediction with Multivariate Regression

In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import yahoofinancials as yf
from yahoo_fin.stock_info import *
import requests_html
import requests
import ftplib
import ta


In [7]:
history = yf.YahooFinancials('ASELS.IS').get_historical_price_data('2019-01-01', '2019-12-31', 'daily')
df = pd.DataFrame(history['ASELS.IS']['prices'])
df.head()

,date,high,low,open,close,volume,adjclose,formatted_date
0,1546327800,12.08,12.08,12.08,12.08,0,11.713909,2019-01-01
1,1546414200,12.05,11.75,12.05,11.77,13489318,11.413302,2019-01-02
2,1546500600,11.81,11.29,11.70,11.34,22491438,10.996334,2019-01-03
3,1546587000,11.51,11.32,11.41,11.48,18335498,11.132091,2019-01-04
4,1546846200,11.61,11.44,11.56,11.46,14899270,11.112698,2019-01-07


In [8]:
df.drop('date', axis=1, inplace=True)